### Assignment 2, Demo
## Support Vector Classifier

Given a dataset with each datapoint representing a tumor our goal is to classify these tumors as either malignant or benign. Each datapoint has 30 feature attributes.

The classification is done by a Support Vector Machine which aims to separate the two classes by a hyperplane in order to minimize the loss or misclassification error. In a perfect scenario all datapoints belonging to each respective class are situated on their own side of the two lines. This would mean that the loss is equal to zero.

This is formulated as the following minimization problem.

minimize $f(w, b) = \frac{1}{2}w^Tw + C\sum^{m}_{i=1}\epsilon_i$

subject to $y_i(w^Tx_i+ b) \geq 1-\epsilon_i, i = 1...m,  \epsilon_i \geq 0$

In [3]:
import pandas as pd
import numpy as np
#import jupyter_black

#jupyter_black.load()
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from scipy import optimize

In [4]:
# Load dataframe with breast cancer data

columns = list(np.arange(0, 32, 1))
df = pd.read_csv("wdbc.data.csv", names=columns)
df

0  1      2      3       4       5        6        7        8   \
0      842302  M  17.99  10.38  122.80  1001.0  0.11840  0.27760  0.30010   
1      842517  M  20.57  17.77  132.90  1326.0  0.08474  0.07864  0.08690   
2    84300903  M  19.69  21.25  130.00  1203.0  0.10960  0.15990  0.19740   
3    84348301  M  11.42  20.38   77.58   386.1  0.14250  0.28390  0.24140   
4    84358402  M  20.29  14.34  135.10  1297.0  0.10030  0.13280  0.19800   
..        ... ..    ...    ...     ...     ...      ...      ...      ...   
564    926424  M  21.56  22.39  142.00  1479.0  0.11100  0.11590  0.24390   
565    926682  M  20.13  28.25  131.20  1261.0  0.09780  0.10340  0.14400   
566    926954  M  16.60  28.08  108.30   858.1  0.08455  0.10230  0.09251   
567    927241  M  20.60  29.33  140.10  1265.0  0.11780  0.27700  0.35140   
568     92751  B   7.76  24.54   47.92   181.0  0.05263  0.04362  0.00000   

          9   ...      22     23      24      25       26       27      28  \
0    0.14710  ...  25.380  17.33  184.60  2019.0  0.16220  0.66560  0.7119   
1    0.07017  ...  24.990  23.41  158.80  1956.0  0.12380  0.18660  0.2416   
2    0.12790  ...  23.570  25.53  152.50  1709.0  0.14440  0.42450  0.4504   
3    0.10520  ...  14.910  26.50   98.87   567.7  0.20980  0.86630  0.6869   
4    0.10430  ...  22.540  16.67  152.20  1575.0  0.13740  0.20500  0.4000   
..       ...  ...     ...    ...     ...     ...      ...      ...     ...   
564  0.13890  ...  25.450  26.40  166.10  2027.0  0.14100  0.21130  0.4107   
565  0.09791  ...  23.690  38.25  155.00  1731.0  0.11660  0.19220  0.3215   
566  0.05302  ...  18.980  34.12  126.70  1124.0  0.11390  0.30940  0.3403   
567  0.15200  ...  25.740  39.42  184.60  1821.0  0.16500  0.86810  0.9387   
568  0.00000  ...   9.456  30.37   59.16   268.6  0.08996  0.06444  0.0000   

         29      30       31  
0    0.2654  0.4601  0.11890  
1    0.1860  0.2750  0.08902  
2    0.2430  0.3613  0.08758  
3    0.2575  0.6638  0.17300  
4    0.1625  0.2364  0.07678  
..      ...     ...      ...  
564  0.2216  0.2060  0.07115  
565  0.1628  0.2572  0.06637  
566  0.1418  0.2218  0.07820  
567  0.2650  0.4087  0.12400  
568  0.0000  0.2871  0.07039  

[569 rows x 32 columns]

In [5]:
# Encode class column as -1 / 1 to fit SVM algorithm
# -1 is malignant and 1 is benign

df[1] = np.where(df[1] == "M", -1, 1)

In [6]:
# Shuffle data in the data frame before train / test split

df_shuffled = df.sample(frac=1)

In [7]:
# Create 569 x 1 class label array
# Split into train and test sets

Y = df_shuffled[1].to_numpy()
Y_train = Y[:500]
Y_test = Y[500:]

In [8]:
# Create 569 x 30 covariate matrix
# Each column represents one observation with 30 covariates

X = df_shuffled.drop(columns=[0, 1]).to_numpy()

# Perform normalization to speed up computations
X = Normalizer().fit_transform(X)

# Split into train and test sets
X_train = X[:500].T
X_test = X[500:].T

In [9]:
# Create 30 x 1 weight vector with initial weights
# Also create 569 x 1 intercept / bias vector with initial value

W = np.random.normal(loc=0, scale=0.1, size=X_train.shape[0])
b_train = np.ones(X_train.shape[1])
b_test = np.ones(X_test.shape[1])

In [10]:
# Transform class label vector into triangular matrix with labels on diagonal

Y_triang = np.zeros((X_train.shape[1], X_train.shape[1]))
np.fill_diagonal(Y_triang, Y_train)

In [11]:
# Initialize 569 x 1 error / residual vector, and penalty term C

# epsilon_vec = np.zeros(X.shape[1])
C = 1000

In [12]:
# Check that we have right shapes

Y_train.shape, Y_test.shape, Y_triang.shape, X_train.shape, X_test.shape, W.shape, b_train.shape, b_test.shape

((500,), (69,), (500, 500), (30, 500), (30, 69), (30,), (500,), (69,))

In [13]:
def calculate_value_per_obs(W):

    # Predicted value for each x_i with current weights, times actual class label
    pred_actual_val = (W.T @ X_train + b_train) @ Y_triang

    return pred_actual_val

In [14]:
def calculate_epsilon(W):

    # Predicted value for each x_i with current weights, times actual class label
    pred_actual_val = calculate_value_per_obs(W)

    # Epsilon value for each x_i on the wrong side of support vector
    epsilon_vec = np.where(pred_actual_val < 1, 1 - pred_actual_val, 0)

    return epsilon_vec

In [15]:
def objective_function(W):  # , C, epsilon_vec):

    # Call function to calculate value per observation
    pred_actual_val = calculate_value_per_obs(W)

    # Call function to calculate current epsilon vector
    epsilon_vec = calculate_epsilon(W)

    # Calculate value of function for current weights and deviations
    function_val = 0.5 * (W.T @ W) + C * np.sum(epsilon_vec)

    return function_val

In [16]:
def constraint_function(W):  # , X, b, Y_triang):

    # Call function to calculate value per observation
    pred_actual_val = calculate_value_per_obs(W)

    # Call function to calculate current epsilon vector
    epsilon_vec = calculate_epsilon(W)

    # Difference between these
    constraint = pred_actual_val - 1 + epsilon_vec

    return constraint

In [17]:
# Set up and run the optimization

epsilon_cons = {"type": "ineq", "fun": constraint_function}

optim = optimize.minimize(
    fun=objective_function,
    x0=W,
    method="SLSQP",
    constraints=epsilon_cons,
    options={"maxiter": 1000},
)

# Store final 
W_final = optim.x
optim

     fun: 90598.70736755287
     jac: array([ 1.52441406e+00,  2.59082031e+01,  7.57373047e+01,  5.66689453e+02,
       -3.32031250e-02, -1.74804688e-01, -8.00781250e-02, -6.93359375e-02,
       -1.95312500e-03,  3.90625000e-03, -1.97265625e-01,  7.00195312e-01,
       -1.96484375e+00, -1.97988281e+01, -1.95312500e-03, -5.95703125e-02,
       -2.73437500e-02, -4.58984375e-02, -5.17578125e-02,  1.46484375e-02,
        1.54492188e+00,  3.15996094e+01,  8.57226562e+01,  7.28445312e+02,
        5.85937500e-03, -1.89453125e-01, -2.37304688e-01, -6.34765625e-02,
        7.22656250e-02,  1.07421875e-02])
 message: 'Optimization terminated successfully'
    nfev: 6063
     nit: 193
    njev: 190
  status: 0
 success: True
       x: array([ 2.79755766e+01,  1.09143409e+01,  7.92236452e+01,  5.43859357e+00,
       -5.32742471e-01, -2.88766575e+00, -4.05695225e+00, -1.72736197e+00,
       -8.54836591e-01, -1.29573946e-01,  8.17790181e-01, -1.59500235e+00,
       -6.16485824e+00, -1.02471650e+01, 

In [18]:
# Make predictions on test and train set

pred_test = np.where(W_final @ X_test >= 0, 1, -1)
pred_train = np.where(W_final @ X_train >= 0, 1, -1)

In [19]:
# Evaluate accuracy of predictions

test_accuracy = np.mean(pred_test == Y_test)
train_accuracy = np.mean(pred_train == Y_train)
test_accuracy, train_accuracy

(0.8840579710144928, 0.884)

In [20]:
# Compare to sklearn SVC estimator

target = df[1].to_numpy()
features = df.drop(columns=[0, 1]).to_numpy()
features = Normalizer().fit_transform(features)

x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=69)

svc = SVC()
svc.fit(x_train, y_train)
pred_test_skl = svc.predict(x_test)
pred_train_skl = svc.predict(x_train)

test_accuracy_skl = np.mean(pred_test_skl == y_test)
train_accuracy_skl = np.mean(pred_train_skl == y_train)
test_accuracy_skl, train_accuracy_skl

(0.8115942028985508, 0.902)